## This notebook was only created to test the ReadAndRunAllWorkFlows.py


<br>
<br>

In [1]:
#pull the ara results and look through KF+G edge graphs and pull the povenance attribute and make a table that will look like
#rows will be kp
#col be workflows
#entry would be the ara's that had edges in kp in the workflow. 

In [1]:
import requests
import json
from collections import defaultdict
import os
import pandas as pd
import numpy as np


#https://ars.ci.transltr.io/ars/api --- 
#https://arax.ci.transltr.io -- backup url

In [2]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

In [3]:
def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' 
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

<br>

### Submiting more than one jobs (test)

<br>

In [ ]:
urls = ['/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.9_EGFR_advanced.json', '/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json']

In [ ]:
dict_workflows = {}
for i in urls:
    feature = (os.path.splitext(os.path.basename(i))[0])
    print(i, feature)
    with open(i,'r') as inf:
        query = json.load(inf)
        kcresult = submit_to_ars(query)
        #xx = retrieve_devars_results(kcresult)
        
        dict_workflows[feature] = kcresult

In [ ]:
dict_workflows

In [ ]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

In [ ]:
workflow_result_messages

In [ ]:
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
        if key.startswith('kp-'):
            key_mod = key.replace('kp-','')
        else:
            key_mod = key
        
        final_dict[key_mod].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [ ]:
df

In [ ]:
df.loc['pk_id']

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
pwd

<br>

## Test for individual run

<br>

In [13]:
with open('/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.3b_DILI-two-hop-drug.json') as inf:
    query1 = json.load(inf)

In [14]:
printjson(query1)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "ids": [
                        "MONDO:0005359",
                        "SNOMEDCT:197354009"
                    ],
                    "categories": [
                        "biolink:DiseaseOrPhenotypicFeature"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:DiseaseOrPhenotypicFeature"
                    ]
                },
                "n2": {
                    "categories": [
                        "biolink:Drug"
                    ]
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1",
                    "predicates": [
                        "biolink:has_real_world_evidence_of_association_with"
                    ]
                },
                "e02": {
                

In [15]:
kcresult = submit_to_ars(query1)

https://arax.ncats.io/?source=ARS&id=cf309e82-c64a-4a10-993d-6c34f0b9d7be


In [10]:
kcresult

'adbacc40-c385-4f41-b1e0-81ed2317df1d'

In [12]:
retrieve_ars_results(kcresult)

Running


{}

In [ ]:
kcresult = 'dc616d70-18d9-47b4-90c2-1abc735398ae'
ars_url='https://ars-dev.transltr.io/ars/api'
pk = 'https://arax.ncats.io/?source=ARS&id=' + kcresult
message_url = f'{ars_url}/messages/{kcresult}?trace=y'
response = requests.get(message_url)
j = response.json()

def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

#check_sheet.set_index('Workflow', inplace=True, drop=True)
name = 'D.1_parkinsons-crohns.json'

In [2]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [3]:
if '.json' in list(check_sheet.Workflow):
    print('True')

True


In [7]:
check_sheet[check_sheet['Workflow']== 'C.1a_SmallMolecule_real_world_evidence_MultSclerosis.json']

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
8,C.1a_SmallMolecule_real_world_evidence_MultScl...,MONDO:0005301,100.0,n03,include,NaN,NaN


In [9]:
if 'C.1a_SmallMolecule_real_world_evidence_MultSclerosis.json' in list(check_sheet.Workflow):
    dfy = check_sheet[check_sheet['Workflow']== 'C.1a_SmallMolecule_real_world_evidence_MultSclerosis.json']
dfy.reset_index(drop=True)

for index,val in enumerate(dfy.Curie):
    print(index,val)
    node_num = dfy.iloc[index][3]
    query_id = val
    if np.isnan(dfy.iloc[index][2]) == True:
        len_check = 2000
    else:
        len_check = dfy.iloc[index][2]

    print(index, query_id, node_num, len_check)

0 MONDO:0005301
0 MONDO:0005301 n03 100.0


In [10]:
dfy

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
8,C.1a_SmallMolecule_real_world_evidence_MultScl...,MONDO:0005301,100.0,n03,include,NaN,NaN


In [19]:
kcresult = 'b5e37dbf-d4e1-402a-9dd1-dda0f52a24de'
ars_url='https://ars.transltr.io/ars/api'
pk = 'https://arax.ncats.io/?source=ARS&id=' + kcresult
message_url = f'{ars_url}/messages/{kcresult}?trace=y'
response = requests.get(message_url)
j = response.json()

def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

#check_sheet.set_index('Workflow', inplace=True, drop=True)
name = 'D.1_parkinsons-crohns.json'


results = {}
dictionary = {}
dictionary_2 = {}
dict3 = {}
for child in j['children']:
    print(child['status'])
    error_code = child['code']

    if child['status']  == 'Done':
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        try:
            
            child_response = requests.get(child_url).json()
            nresults = len(child_response['fields']['data']['message']['results'])
            
            if nresults > 0:
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                
            
                if name in list(check_sheet.Workflow):
                    #print(name)
                    
                
                    dfy = check_sheet[check_sheet['Workflow']== name]

                    dfy.reset_index(drop=True)
                    

                    
                    for index,curie_id in enumerate(dfy.Curie):
                        print(index,curie_id)
                        node_num = dfy.iloc[index][3]
                        query_id = curie_id
                        if np.isnan(dfy.iloc[index][2]) == True:
                            len_check = len(child_response['fields']['data']['message']['results'])
                            len_check = int(len_check)
                        else:
                            len_check = dfy.iloc[index][2]
                            len_check = int(len_check)

                        print(node_num, query_id, len_check)
                        
                        

                        locs = []
                        for x, val in enumerate(child_response['fields']['data']['message']['results']):
                            #print(val)

                            if x < len_check:

                                if query_id in val['node_bindings'][node_num][0]['id']:
                                    locs.append(x)
                        if not locs:
                            check_result = f'False'
                            print(check_result)
                            #pass
                        else:
                            check_result = f'True'
                            print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                        dict3[curie_id] = check_result

                        print(dict3)
                        
                    
            
            if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                        edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                        test_att_values =[]
                        for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            #print(val)

                            for tx in edge_ex[val]['attributes']:
                                if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :


                                    value_att = tx['value']

                                    test_att_values.append(value_att)
                                    test_att = set(flatten_list(test_att_values))


                                    dictionary_2[child['actor']['agent']] = test_att
                #else:
                    #dictionary_2[child['actor']['agent']] = [] 
            #else:
               # dictionary_2[child['actor']['agent']] = []

        except Exception as e:
            nresults=0
            child['status'] = 'ARS Error'
            #dictionary_2[child['actor']['agent']] = []



    elif child['status'] == 'Error':
        nresults=0
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        try:
            child_response = requests.get(child_url).json()
            results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            #dictionary_2[child['actor']['agent']] = []
        except Exception as e:
            #print(e)
            child['status'] = 'ARS Error'
            #dictionary_2[child['actor']['agent']] = []


    else:
        nresults = 0
        #dictionary_2[child['actor']['agent']] = []

    dictionary['pk_id'] =  pk  

    if ((child['status'] == 'Done') & (nresults == 0)):
        dictionary[child['actor']['agent']] = 'No Results: ' + str(error_code)
        #test =  [child['actor']['agent'], 'No Results']
    elif ((child['status'] == 'ARS Error') & (nresults == 0)):
        dictionary[child['actor']['agent']] = 'ARS Error: ' + str(error_code)
    elif ((child['status'] == 'Error') & (nresults == 0)):
        dictionary[child['actor']['agent']] = 'Error: ' + str(error_code)
        #test =  [child['actor']['agent'], 'ARS Error']
    elif ((child['status'] == 'Done') & (nresults != 0)):
        #test =  [child['actor']['agent'], 'Results']
        dictionary[child['actor']['agent']] = 'Results: ' + str(error_code) + str(dict3)
    elif ((child['status'] == 'Unknown') & (nresults == 0)):
        #test =  [child['actor']['agent'], 'Results']
        dictionary[child['actor']['agent']] = 'Unknown: ' + str(error_code) 

    print(child['actor']['agent'], child['status'], nresults)
    #test =  [child['actor']['agent'], child['status'], nresults]
    #test2.append(test)

Error
ara-unsecret Error 0
Done
kp-icees-dili ARS Error 0
Error
ara-robokop Error 0
Done
kp-cam Done 0
Error
kp-molecular Error 0
Done
kp-icees ARS Error 0
Error
ara-bte Error 0
Done
ara-explanatory Done 0
Done
kp-openpredict Done 0
Unknown
ara-ncats Unknown 0
Done
0 NCBIGene:120892
n01 NCBIGene:120892 22
curie id: NCBIGene:120892 : INCLUDED at postion N == [0] on n01
{'NCBIGene:120892': 'True'}
1 NCBIGene:11315
n01 NCBIGene:11315 22
curie id: NCBIGene:11315 : INCLUDED at postion N == [2] on n01
{'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True'}
2 NCBIGene:110357
n01 NCBIGene:110357 22
False
{'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}
ara-improving Done 22
Error
kp-cohd Error 0
Done
0 NCBIGene:120892
n01 NCBIGene:120892 9007
curie id: NCBIGene:120892 : INCLUDED at postion N == [9, 22, 24, 27, 28, 62, 66, 68, 91, 95, 204, 208, 228, 236, 245, 247, 249, 283, 303, 309, 310, 314, 317, 336, 338, 342, 343, 422, 431, 440, 441, 450, 618, 703, 725, 778, 7

In [ ]:
dictionary_2

In [20]:
dictionary

{'pk_id': 'https://arax.ncats.io/?source=ARS&id=b5e37dbf-d4e1-402a-9dd1-dda0f52a24de',
 'ara-unsecret': 'Error: 598',
 'kp-icees-dili': 'ARS Error: 200',
 'ara-robokop': 'Error: 500',
 'kp-cam': 'No Results: 200',
 'kp-molecular': 'Error: 501',
 'kp-icees': 'ARS Error: 200',
 'ara-bte': 'Error: 598',
 'ara-explanatory': 'No Results: 200',
 'kp-openpredict': 'No Results: 200',
 'ara-ncats': 'Unknown: 503',
 'ara-improving': "Results: 200{'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}",
 'kp-cohd': 'Error: 400',
 'ara-aragorn': "Results: 200{'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}",
 'kp-chp': 'No Results: 200',
 'ara-aragorn-exp': 'Error: 404',
 'ara-arax': 'Error: 598',
 'kp-textmining': 'No Results: 200',
 'kp-genetics': 'Unknown: 503'}

In [ ]:
dict3

In [13]:
name = 'D.1_parkinsons-crohns.json'

In [22]:
kcresult = 'dc616d70-18d9-47b4-90c2-1abc735398ae'
ars_url='https://ars-dev.transltr.io/ars/api'

pk = 'https://arax.ncats.io/?source=ARS&id=' + kcresult
message_url = f'{ars_url}/messages/{kcresult}?trace=y'
response = requests.get(message_url)
j = response.json()
print( j['status'] )
results = {}
dictionary = {}
dictionary_2 = {}
dict3 = {}
for child in j['children']:
    print(child['status'])
    error_code = child['code']

    if child['status']  == 'Done':
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        try:
            
            child_response = requests.get(child_url).json()
            nresults = len(child_response['fields']['data']['message']['results'])
            
            if nresults > 0:
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                
            
                if name in list(check_sheet.Workflow):
                    dfy = check_sheet[check_sheet['Workflow']== name]

                dfy.reset_index(drop=True)



                for index,curie_id in enumerate(dfy.Curie):
                    print(index,curie_id)
                    node_num = dfy.iloc[index][3]
                    query_id = curie_id
                    if np.isnan(dfy.iloc[index][2]) == True:
                        len_check = len(child_response['fields']['data']['message']['results'])
                    else:
                        len_check = dfy.iloc[index][2]

                    #print(node_num, query_id, len_check)

                    locs = []
                    for x, val in enumerate(child_response['fields']['data']['message']['results']):
                        print(val)

                        if x < len_check:

                            if query_id in val['node_bindings'][node_num][0]['id']:
                                locs.append(x)
                    if not locs:
                        check_result = f'False'
                        print(check_result)
                        #pass
                    else:
                        check_result = f'True'
                        print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                    dict3[curie_id] = check_result
            #else:
                #continue
            
            if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                        edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                        test_att_values =[]
                        for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            #print(val)

                            for tx in edge_ex[val]['attributes']:
                                if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :


                                    value_att = tx['value']

                                    test_att_values.append(value_att)
                                    test_att = set(flatten_list(test_att_values))


                                    dictionary_2[child['actor']['agent']] = test_att
                #else:
                    #dictionary_2[child['actor']['agent']] = [] 
            #else:
               # dictionary_2[child['actor']['agent']] = []

        except Exception as e:
            nresults=0
            child['status'] = 'ARS Error'
            #dictionary_2[child['actor']['agent']] = []



    elif child['status'] == 'Error':
        nresults=0
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        try:
            child_response = requests.get(child_url).json()
            results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            #dictionary_2[child['actor']['agent']] = []
        except Exception as e:
            #print(e)
            child['status'] = 'ARS Error'
            #dictionary_2[child['actor']['agent']] = []


    else:
        nresults = 0
        #dictionary_2[child['actor']['agent']] = []

    dictionary['pk_id'] =  pk  

    if ((child['status'] == 'Done') & (nresults == 0)):
        dictionary[child['actor']['agent']] = 'No Results: ' + str(error_code)
        #test =  [child['actor']['agent'], 'No Results']
    elif ((child['status'] == 'ARS Error') & (nresults == 0)):
        dictionary[child['actor']['agent']] = 'ARS Error: ' + str(error_code)
    elif ((child['status'] == 'Error') & (nresults == 0)):
        dictionary[child['actor']['agent']] = 'Error: ' + str(error_code)
        #test =  [child['actor']['agent'], 'ARS Error']
    elif ((child['status'] == 'Done') & (nresults != 0)):
        #test =  [child['actor']['agent'], 'Results']
        dictionary[child['actor']['agent']] = 'Results: ' + str(error_code) + str(dict3)
    elif ((child['status'] == 'Unknown') & (nresults == 0)):
        #test =  [child['actor']['agent'], 'Results']
        dictionary[child['actor']['agent']] = 'Unknown: ' + str(error_code) 

    print(child['actor']['agent'], child['status'], nresults)

Done
Done
ara-aragorn ARS Error 0
Done
0 NCBIGene:120892
{'node_bindings': {'n02': [{'id': 'DOID:8778'}], 'n00': [{'id': 'MONDO:0011658'}], 'n01': [{'id': 'UniProtKB:Q99497'}]}, 'edge_bindings': {'e01': [{'id': 'RTX-KG2:UniProtKB:Q99497-biolink:gene_associated_with_condition-MONDO:0005011', 'attributes': None}, {'id': 'GeneticsKP:UniProtKB:Q99497-biolink:gene_associated_with_condition-DOID:8778', 'attributes': None}], 'N3': [{'id': 'N3_2301', 'attributes': None}], 'N1': [{'id': 'N1_346', 'attributes': None}], 'e00': [{'id': 'RTX-KG2:MONDO:0011658-biolink:has_participant-UniProtKB:Q99497', 'attributes': None}, {'id': 'RTX-KG2:UniProtKB:Q99497-biolink:related_to-MONDO:0011658', 'attributes': None}, {'id': 'RTX-KG2:MONDO:0011658-biolink:related_to-UniProtKB:Q99497', 'attributes': None}, {'id': 'RTX-KG2:UniProtKB:Q99497-biolink:close_match-MONDO:0011658', 'attributes': None}, {'id': 'RTX-KG2:UniProtKB:Q99497-biolink:gene_associated_with_condition-MONDO:0011658', 'attributes': None}], 'N2':

ara-arax Done 500
Done
0 NCBIGene:120892
{'node_bindings': {'n00': [{'id': 'MONDO:0005180'}], 'n01': [{'id': 'NCBIGene:1490'}], 'n02': [{'id': 'MONDO:0005011'}]}, 'edge_bindings': {'e00': [{'id': 'e6f03cac1ec27ceec33d994b10644f39'}, {'id': '9ba3f30d5dd05cf1d001eca90cc2591d'}, {'id': '17db2ead6fed49ebc719666ee102e14e'}], 'e01': [{'id': '44fc063e5a8dd3ce7b7e17cabdc5c71c'}]}, 'score': 1}
{'node_bindings': {'n00': [{'id': 'MONDO:0005180'}], 'n01': [{'id': 'NCBIGene:3123'}], 'n02': [{'id': 'MONDO:0005011'}]}, 'edge_bindings': {'e00': [{'id': '5c0cb3b5b80f93a68df08203659da776'}], 'e01': [{'id': '81febafabb19ef6efac835d71dff14a4'}, {'id': '87d75ea130078fa5ee2dbe162dc50dfc'}, {'id': 'd171dd35dec86a54168165b9ce6e1ad5'}, {'id': '60a987da2880c673616df24f775822db'}]}, 'score': 1}
{'node_bindings': {'n00': [{'id': 'MONDO:0005180'}], 'n01': [{'id': 'NCBIGene:3635'}], 'n02': [{'id': 'MONDO:0005011'}]}, 'edge_bindings': {'e00': [{'id': '4d0c6f92d3ceaa3d4214e33166395709'}, {'id': '16b12bf12fbbad8e5b972

ara-bte Done 556


In [21]:
val

'00b64e7f-ed71-4c93-86c9-e9c925720b94'

In [15]:
dictionary

{'pk_id': 'https://arax.ncats.io/?source=ARS&id=dc616d70-18d9-47b4-90c2-1abc735398ae',
 'ara-aragorn': 'ARS Error: 200',
 'ara-arax': "Results: 200{'NCBIGene:120892': 'False', 'NCBIGene:11315': 'False', 'NCBIGene:110357': 'False'}",
 'ara-bte': "Results: 200{'NCBIGene:120892': 'True', 'NCBIGene:11315': 'True', 'NCBIGene:110357': 'False'}"}

In [ ]:
def retrieve_ars_results(mid, name, check_sheet, ars_url='https://ars-dev.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    dict3 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']

        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:

                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])

                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}


                    if name in list(check_sheet.Workflow):
                        print(name)
                        dfy = check_sheet[check_sheet['Workflow']== name]

                    dfy.reset_index(drop=True)



                    for index,curie_id in enumerate(dfy.Curie):
                        print(index,curie_id)
                        node_num = dfy.iloc[index][3]
                        query_id = curie_id
                        if np.isnan(dfy.iloc[index][2]) == True:
                            len_check = len(child_response['fields']['data']['message']['results'])
                        else:
                            len_check = dfy.iloc[index][2]

                        #print(node_num, query_id, len_check)

                        locs = []
                        for x, val in enumerate(child_response['fields']['data']['message']['results']):
                            #print(val)

                            if x < len_check:

                                if query_id in val['node_bindings'][node_num][0]['id']:
                                    locs.append(x)
                        if not locs:
                            check_result = f'False'
                            print(check_result)
                            #pass
                        else:
                            check_result = f'True'
                            print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                        dict3[curie_id] = check_result
                #else:
                    #continue

                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)

                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :


                                        value_att = tx['value']

                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))


                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []

            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []



        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []


        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []

        dictionary['pk_id'] =  pk  

        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results: ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error: ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error: ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results: ' + str(error_code) + str(dict3)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown: ' + str(error_code) 

        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]

In [ ]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [ ]:
dict_workflows = {'A.0_RHOBTB2_direct': '9a344e22-3ad1-4f83-bb81-b1c771142c83','D.1_parkinsons-crohns': 'dc616d70-18d9-47b4-90c2-1abc735398ae'}

In [ ]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    name = keys + '.json'
    print(name, val)
    
    
    result_status = retrieve_ars_results(val, name, check_sheet)
    
    workflow_result_messages[keys] = result_status

In [ ]:
col = []
final_dict = defaultdict(list)

for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][0].items():
        #print(key, value)
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [ ]:
df

In [ ]:
if 'D.1_parkinsons-crohns.json' in list(check_sheet.Workflow):
    dfy = check_sheet[check_sheet['Workflow']== 'D.1_parkinsons-crohns.json']
dfy.reset_index(drop=True)

for index,val in enumerate(dfy.Curie):
    print(index,val)
    node_num = dfy.iloc[index][3]
    query_id = val
    if np.isnan(dfy.iloc[index][2]) == True:
        len_check = 2000
    else:
        len_check = dfy.iloc[index][2]

    print(index, query_id, node_num, len_check)

In [ ]:
dfy